<h1> Emoji DB for James </h1>

In [1]:
from __future__ import division
import json
import pandas as pd
import re, collections
import numpy as np
import os
base_dir=os.path.expanduser('~')
import datetime
import time
import psycopg2
from nltk.corpus import words

<h1> Connect to the emoji_db </h1>

In [3]:
#connect
conn = psycopg2.connect("host=52.38.112.55 port=5432 dbname=emoji_db user=WizzKid password=IVapeBro")
cur = conn.cursor()

KeyboardInterrupt: 

In [272]:
#connect
conn = psycopg2.connect("host=localhost port=5432 dbname=emoji_db user=postgres password=darkmatter")
cur = conn.cursor()

<h1> Loop through the english dictionary </h1>

In [ ]:
lang='en'
facefilter='on'
for word in words.words():
    print(word)
    word=word.lower()
    word=word.replace("'","''")#replace all apostrophes with double for SQL query
    cur.execute("SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '%{:s}%' AND lang='{:s}' )) as T WHERE(T.FF is True) group by T.Label order by TFreq DESC limit 15;".format(word,lang))
    result=self.cur.fetchall()
    xdata=[val[0] for val in result]
    ydata=[val[1] for val in result]

In [ ]:
pattern_type = 'single'
freq_filter = 'all'
face_filter = 'on'
user_lang = 'en'
for word in words.words():
    print(word)
    if freq_filter=='freq':
        xdata, ydata = Emoji.filter_emoji_freq(word,face_filter,pattern_type,user_lang)
    elif freq_filter=='all':
        xdata, ydata = Emoji.filter_emoji(word,face_filter,pattern_type,user_lang)
    else: #surr (surrounding text, takes long to query)
        xdata, ydata = Emoji.filter_emoji_surr(word,face_filter,pattern_type,user_lang)
    #write result to DB
    Emoji.index_result(word,freq_filter,face_filter,pattern_type,user_lang,xdata,ydata)

<h1> select the top word in one language and match to another </h1>

In [336]:
word='snow'
cur.execute("select DISTINCT(b.searchTerm), (SELECT COUNT(*) from (select unnest(a.emojiLabel[1:4]) as yp intersect select unnest(b.emojiLabel[1:4])as yo) as Foo) \
from (select emojiLabel from emoji_search where searchterm=' {:s}' AND \
lang='en' AND facefilter='on' order by date DESC limit 1) a join emoji_search b on \
(select COUNT(*) from (select unnest(a.emojiLabel[1:4]) as yp intersect select unnest(b.emojiLabel[1:4])as yo) as Foo)>=2 \
WHERE b.lang='es' AND b.facefilter='on' ;".format(word))

In [337]:
result=cur.fetchall()
result

[(' dec\xc3\xada', 2L),
 (' nieve', 2L),
 (' llover', 2L),
 (' hielo', 2L),
 (' alaska', 2L),
 (' dec\xc3\xadan', 2L)]

In [295]:
for item in result[0]:
    print result

[('\xe8\xa1\x97\xe9\x81\x93', 2L)]
[('\xe8\xa1\x97\xe9\x81\x93', 2L)]


In [329]:
print('\xeb\x8d\x94')

더


<h1> Search by top result </h1>

In [334]:
word='snow'
cur.execute("select b.searchTerm from (select emojiLabel from emoji_search where searchterm=' {:s}' AND \
lang='en' AND facefilter='on' order by date DESC limit 1) a join emoji_search b on \
a.emojiLabel[1]=b.emojiLabel[1]\
WHERE b.lang='es' AND b.facefilter='on' ;".format(word))

In [335]:
result=cur.fetchall()
result

[(' dec\xc3\xada',),
 (' fr\xc3\xado',),
 (' hielo',),
 (' nieve',),
 (' fresco',),
 (' dispara',),
 (' pieza',),
 (' dec\xc3\xadan',),
 (' llover',),
 (' empacar',),
 (' cagada',),
 (' cuidarse',),
 (' congelada',),
 (' cambiarse',),
 (' disparate',),
 (' revisar\xc3\xa9',)]

In [327]:
print(result[0][0])

임마


In [191]:
word='beer'
cur.execute("select emojiLabel[1:4] from emoji_search where searchterm=' beer' AND lang='en' AND facefilter='on' order by date DESC limit 1;")